### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
from tqdm.notebook import tqdm
from neuropy.core import Epoch
import seaborn as sns
from neuropy.utils.mathutil import min_max_scaler
import pandas as pd
from neuropy.utils.position_util import run_direction
from neuropy.plotting import plot_epochs
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
import subjects
from scipy import stats

### Comparing experience on the track between NSD and SD

In [ ]:
sessions = subjects.ripple_sess()

maze_df = []
for s, sess in enumerate(sessions):
    maze = sess.paradigm["maze"]
    maze_duration = maze.durations[0] / 3600
    # print(sess)
    # print(sess.maze_run.n_epochs)

    df = pd.DataFrame(
        dict(maze_duration=maze_duration, session=s, grp=sess.tag, index=[0])
    )
    maze_df.append(df)

maze_df = pd.concat(maze_df, ignore_index=True)

In [ ]:
x = maze_df[maze_df.grp == "NSD"].maze_duration.values
y = maze_df[maze_df.grp == "SD"].maze_duration.values

stats.ttest_ind(x, y, equal_var=False)

In [ ]:
from plotters import Plotter


fig = subjects.SdFig().fig_supp(ncols=10)
ax = fig.subplot(fig.gs[0])

sns.stripplot(
    data=maze_df,
    x="grp",
    y="maze_duration",
    ax=ax,
    size=3,
    edgecolor="w",
    linewidth=0.3,
)
sns.barplot(
    data=maze_df,
    x="grp",
    y="maze_duration",
    ax=ax,
    ci=None,
    facecolor="w",
    edgecolor="0.2",
)

ax.set_ylim(0.5, 1.3)
ax.set_ylabel("MAZE duration (h)")

fig.savefig(subjects.figpath_sd / "maze_duration_sd_vs_nsd", format="svg")

### Scratch/Test for running epochs

In [ ]:
sess = subjects.nsd.ratKday2[0]

maze = sess.paradigm["maze"].flatten()
t = sess.maze.time
dt = 1 / sess.maze.sampling_rate
x = gaussian_filter1d(sess.maze.x, sigma=0.1 / dt)
speed = np.abs(np.concatenate([[0], np.diff(x) / dt]))

speed_thresh = np.where(speed >= 10, speed, 0)
peaks, props = find_peaks(speed_thresh, height=30, prominence=0)

starts, stops = props["left_bases"], props["right_bases"]
peaks_power = speed_thresh[peaks]

# ----- merge overlapping epochs ------
n_epochs = len(starts)
ind_delete = []
for i in range(n_epochs - 1):
    if (starts[i + 1] - stops[i]) < (1 * (1 / dt) + 1e-6):
        # stretch the second epoch to cover the range of both epochs
        starts[i + 1] = min(starts[i], starts[i + 1])
        stops[i + 1] = max(stops[i], stops[i + 1])

        peaks_power[i + 1] = max(peaks_power[i], peaks_power[i + 1])
        peaks[i + 1] = [peaks[i], peaks[i + 1]][
            np.argmax([peaks_power[i], peaks_power[i + 1]])
        ]

        ind_delete.append(i)

epochs_arr = np.vstack((starts, stops, peaks, peaks_power)).T
starts, stops, peaks, peaks_power = np.delete(epochs_arr, ind_delete, axis=0).T
starts = starts.astype("int")
stops = stops.astype("int")

run = run_direction(sess.maze, min_distance=10, sigma=0.1)
peak_time = run.to_dataframe().peak_time.values
_, axs = plt.subplots(2, 1, sharex=True)

axs[0].plot(t, x)
# ax.plot(sess.lin_maze.time,gaussian_filter1d(sess.lin_maze.x,sigma=50))
plot_epochs(
    ax=axs[0],
    epochs=run,
    colors={"up": "r", "down": "k"},
    alpha=0.2,
    collapsed=True,
)

axs[1].plot(t, speed)
axs[1].plot(t[starts], speed[starts], "r*")
axs[1].plot(t[stops], speed[stops], "k*")
axs[1].plot(peak_time, 60 * np.ones_like(peak_time), "g*")

# axs[2].plot(t[1:],np.abs(np.diff(x)/dt))

### Match linearized maps for maze and remaze
- The origin of isomap-linearized maps are arbitary, so it is possible the origins are at different ends on maze and remaze.

In [ ]:
from neuropy.core import Position

sessions = subjects.remaze_sess()[1:]

fig = plotting.Fig(4, 4)
for s, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    remaze = sess.paradigm["re-maze"].flatten()

    position = sess.position
    t = position.time
    x = position.x
    y = position.y

    maze_run = sess.maze_run
    remaze_run = sess.remaze_run

    linmaze = sess.maze
    t_linmaze = linmaze.time[linmaze.x < 100]

    if s in [0, 1, 4]:
        linremaze = sess.remaze
        linremaze.x = -1 * linremaze.x + np.max(linremaze.x)
        linremaze.save(sess.filePrefix.with_suffix(".remaze.linear"))
    else:
        linremaze = sess.remaze

    t_linremaze = linremaze.time[linremaze.x < 100]

    origin_x_maze = np.interp(t_linmaze, t, x)
    origin_y_maze = np.interp(t_linmaze, t, y)

    origin_x_remaze = np.interp(t_linremaze, t, x)
    origin_y_remaze = np.interp(t_linremaze, t, y)

    ax = fig.subplot(fig.gs[s])
    ax.plot(origin_x_maze, origin_y_maze, "k.")
    ax.plot(origin_x_remaze, origin_y_remaze, "r.")

In [ ]:
sess.maze_run.to_dataframe()

### Pooled running epochs detection

In [ ]:
# sessions = subjects.pf_sess()
# sessions = subjects.nsd.ratVday3 + subjects.sd.ratUday1
# sessions = subjects.nsd.ratVday1 + subjects.nsd.ratVday3
sessions = subjects.remaze_sess()[1:]
# sessions = subjects.sd.ratVday2
epoch_names = ["maze", "remaze"]

fig = plotting.Fig(8, 2)
for s, sess in enumerate(sessions):
    # maze = sess.paradigm["re-maze"].flatten()

    for e, name in enumerate(epoch_names):
        if name == "maze":
            pos = sess.maze
        if name == "remaze":
            pos = sess.remaze

        run_epochs = run_direction(pos, min_distance=10, sigma=0.1)
        run_epochs.save(sess.filePrefix.with_suffix(f".{name}.running"))
        ax = fig.subplot(fig.gs[s, e])
        ax.plot(pos.time, pos.x)
        plotting.plot_epochs(run_epochs, ax=ax, collapsed=True)

# if hasattr(sess, "maze_run"):
#     print(sess.maze_run.to_dataframe())
# if not hasattr(sess, "remaze_run"):
#     print(sess.remaze_run.to_dataframe())

### Visualize 1d linearize position

In [ ]:
sessions = subjects.pf_sess()
# sessions = subjects.nsd.ratKday2

_, axs = plt.subplots(8, 2)
axs = axs.reshape(-1)

for s, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    try:
        mazepos = sess.maze
    except:
        mazepos = sess.position.time_slice(maze)

    axs[s].plot(mazepos.time, mazepos.x)
    axs[s].set_title(sess.animal.name)

In [ ]:
from datetime import date

"." + date.today().strftime("%d-%m-%y")

### Visualize 2D position

In [ ]:
# sessions = subjects.pf_sess()
sessions = subjects.sd.ratUday1


_, axs = plt.subplots(2, 1)
axs = axs.reshape(-1)

for s, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    # maze[0] = maze[0] + 1
    # maze[1] = maze[1] + 20
    # mazepos = sess.position.time_slice(*maze)
    mazepos = sess.maze.time_slice(*maze)
    all_run_epochs = sess.maze_run
    thresh_run_epochs = run_direction(
        mazepos, min_distance=50, sigma=0.2, speed_thresh=(30, None)
    )
    # thresh_run_epochs = sess.maze_run.duration_slice(min_dur=1.2)

    axs[s].plot(mazepos.time, mazepos.x)
    plotting.plot_epochs(
        all_run_epochs, ax=axs[s], collapsed=True, colors="r", alpha=0.5
    )
    plotting.plot_epochs(
        thresh_run_epochs, ax=axs[s], collapsed=True, colors="g", alpha=0.5
    )
    axs[s].set_title(sess.animal.name)